## 패션 이미지 데이터셋 전처리

In [1]:
# import pandas as pd

# path = 'data/meta/'
# train = pd.read_json(path + 'train_no_dup.json')
# valid = pd.read_json(path + 'valid_no_dup.json')
# test = pd.read_json(path + 'test_no_dup.json')
# blank = pd.read_json(path + 'fill_in_blank_test.json')

## Data Load Example

In [1]:
import pandas as pd
from dataloader.sampling import DataSampler
from dataloader.multimodal_data import MultiModalData
from dataloader.multilstm_data import MultiModalData
import os

base_dir = os.getcwd()
data_dir = os.path.join(base_dir, 'data')
meta_dir = os.path.join(data_dir, 'meta')
image_dir = os.path.join(data_dir, 'images')
sampler = DataSampler(data_path = meta_dir, k=150, test_sampling_ratio=1)
concat_df, question_data = sampler.sample_data()

train_dataset = MultiModalData(concat_df, sampler.category_df, image_dir, mode='train')
valid_dataset = MultiModalData(concat_df, sampler.category_df, image_dir, mode='valid')
test_dataset = MultiModalData(concat_df, sampler.category_df, image_dir, question = question_data, mode='test')

In [2]:
train_dataset[10].keys()

c:\ProgramData\Anaconda3\envs\jin\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


dict_keys(['texts', 'images', 'set_id'])

In [3]:
train_dataset[1]

{'texts': tensor([[ 0.1168, -0.1072, -0.4150,  ...,  0.3625, -0.2905,  0.2135],
         [ 0.0575, -0.0611, -0.5083,  ...,  0.2993, -0.1221,  0.0342],
         [ 0.0458,  0.3501, -0.0540,  ...,  0.3625, -0.1615,  0.0183],
         [ 0.4260,  0.0510, -0.4104,  ...,  0.2598,  0.1517,  0.1289],
         [-0.2642, -0.1250, -0.1735,  ...,  0.3481,  0.0607, -0.1196]],
        device='cuda:0', grad_fn=<ToCopyBackward0>),
 'images': tensor([[-0.3042,  0.0260, -0.0421,  ...,  0.5161,  0.0083,  0.3433],
         [-0.0344, -0.4177,  0.5928,  ...,  0.5952, -0.3606,  0.3208],
         [-0.4397, -0.0482,  0.3816,  ...,  0.3572, -0.0570,  0.1208],
         [ 0.1605,  0.0170, -0.1733,  ...,  0.9746, -0.0967,  0.1057],
         [-0.0327,  0.1245,  0.6440,  ...,  0.4250, -0.1155,  0.0605]],
        device='cuda:0', grad_fn=<ToCopyBackward0>),
 'set_id': 203655321}

## Dataloader

In [17]:
# def custom_collate_fn(batch, test=False):
#     texts = [item["texts"] for item in batch]
#     price = [item["price"] for item in batch]         
#     likes = [item["likes"] for item in batch]         
#     images = [item["images"] for item in batch]        
#     set_id = [item["set_id"] for item in batch]       
       
#     if test:
#         question = [item["question"] for item in batch] 
#         return {
#             "texts": texts,
#             "price": price,
#             "likes": likes,
#             "images": images,
#             "set_id": set_id,
#             "question": question,
#         }
#     else: 
#         return {
#             "texts": texts,
#             "price": price,
#             "likes": likes,
#             "images": images,
#             "set_id": set_id
#         }

In [2]:
from torch.utils.data import Dataset, DataLoader
from AI_MMR.model.multilstm import Multifusion, MultiLSTM
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=4, shuffle=True)

In [3]:
# Dataloader

for batch_idx, batch in enumerate(train_dataloader):
    print(batch.keys())
    break

c:\ProgramData\Anaconda3\envs\jin\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


dict_keys(['texts', 'text_gt', 'prices', 'likes', 'images', 'image_gt', 'set_id', 'valid_idx', 'gt_idx'])


In [4]:
mlp = Multifusion().to(device)
lstm = MultiLSTM(input_size=512, hidden_size=512, num_layers=1, bidirectional=True).to(device)

In [24]:
from AI_MMR.model.multilstm import Multifusion